In [18]:
import json
import re
import pickle

import requests
import pandas as pd

### 演示Github媒体API使用方法

In [ ]:
github_path = 'https://api.github.com/repos/{owner}/{repo}'
Accept = 'application/vnd.github.v3.star+json'
github_app_res = requests.get(github_path.format(owner='ipython', repo='ipython'), headers={'Accept': Accept})
github_app_info = json.loads(github_app_res.text)
github_app_info

In [ ]:
star_path = 'https://api.github.com/repos/{owner}/{repo}/stargazers'
Accept = 'application/vnd.github.v3.star+json'
star_res = requests.get(star_path.format(owner='ipython', repo='ipython'), headers={'Accept': Accept})
star_info = json.loads(star_res.text)
star_info

In [ ]:
github_app_res.headers['link']

### 开始爬取

In [9]:
df = pd.read_pickle('pypi_json.pkl')

In [10]:
for i in df.github:
    if i not in (None, True, False):
        print(type(i))
        break

<class 'tuple'>


In [16]:
def get_star_time(df):
    star_path = 'https://api.github.com/repos/{}/{}/stargazers'
    token = 'a778aeacfadcaae486f97b8c9a560121b4b44711'
    Accept = 'application/vnd.github.v3.star+json'
    star_infos = []
    for package_name, aha in df.github.groupby(level=0):
        for i in aha.values[::-1]:
            if i not in (None, True, False):
                star_res = requests.get(star_path.format(*i), headers={'Accept': Accept, 'Authorization': 'token ' + token})
                msg = json.loads(star_res.text)
                try:
                    star_infos.append(dict(
                        _id=package_name,
                        star_time=[i['starred_at'] for i in msg]
                    ))
                except TypeError:
                    if msg['message'] == 'Not Found':
                        continue
                    elif 'rate limit' in msg['message']:
                        return star_infos
                    else:
                        print(msg)
                        return star_infos
                else:
                    break
    return star_infos

In [13]:
packages = sorted(set(list(zip(*df.index))[0]))

In [37]:
packages.index(star_infos[-1]['_id'])

1293

In [35]:
star_infos = get_star_time(df.loc[packages[packages.index(star_infos[-1]['_id']):10000]])

In [38]:
with open(f"./STAR/star_info_0-{packages.index(star_infos[-1]['_id'])}", 'wb') as f:
    pickle.dump(star_infos, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
star_path = 'https://api.github.com/repos/{}/{}/stargazers'
token = 'a778aeacfadcaae486f97b8c9a560121b4b44711'
Accept = 'application/vnd.github.v3.star+json'
i = 'ipython', 'ipython'
res = requests.get(star_path.format(*i), headers={'Accept': Accept, 'Authorization': 'token ' + token})

In [8]:
res.headers['X-RateLimit-Remaining']

'4268'

In [5]:
try:
    raise ValueError('haha')
except ValueError as e:
    print(type(e.message))

AttributeError: 'ValueError' object has no attribute 'message'

In [30]:
from datetime import datetime, timezone
import time
# datetime.datetime.fromtimestamp(time.time())
datetime.now().astimezone().isoformat()

'2020-11-24T14:14:01.756980+00:00'

In [42]:
f'{datetime.fromtimestamp(time.time()):%l:%M%p on %b %d, %Y}'

' 2:23PM on Nov 24, 2020'

In [38]:
datetime.now()

datetime.datetime(2020, 11, 24, 14, 19, 28, 959498)